[语义分割练习](https://zh.d2l.ai/chapter_computer-vision/semantic-segmentation-and-dataset.html)

In [3]:
%matplotlib inline
import os
import torch
import torchvision
from d2l import torch as d2l

In [4]:
#@save
d2l.DATA_HUB['voc2012'] = (d2l.DATA_URL + 'VOCtrainval_11-May-2012.tar',
                           '4e443f8a2eca6b1dac8a6c57641b67dd40621a49')

voc_dir = d2l.download_extract('voc2012', 'VOCdevkit/VOC2012')